## IO

### 文件读写

- 文本文件
- 二进制文件

In [ ]:
with open('../data/content.txt', 'a', encoding='utf8') as f:
    f.write('Hello, world! \n')

with open('../data/content.txt', 'r', encoding='utf8', errors='ignore') as f:
    print(f.read())

    for line in f.readlines():
        print(line.strip())

# with open('test.jpg', 'rb') as f:
#     f.read()

In [ ]:
import os

os.path.abspath('')

### 文件操作

In [ ]:
import os

# 环境变量
print(os.name, os.uname)
print(os.environ)
print(os.environ.get('PATH'))

# 目录
basepath = os.path.abspath('')
realPath = os.path.join(basepath, 'testdir')

if os.path.isdir(realPath) == False:
    os.mkdir(realPath)

print(os.path.split(realPath))
print(os.path.splitext(realPath))

if os.path.isdir((realPath)) == True:
    os.rmdir(realPath)

# 文件
os.rename('tests.txt', 'tests.py')
os.remove('tests.py')

print([x for x in os.listdir('../') if os.path.isdir(x)])
print([x for x in os.listdir('') if os.path.isfile(
    x) and os.path.splitext(x)[1] == '.py'])

### StringIO

In [ ]:
from io import StringIO

f = StringIO()
f.write('hello')
f.write(' ')
f.write('world!')
f.getvalue()

### BytesIO

In [ ]:
from io import BytesIO

f = BytesIO()
f.write('中文'.encode('utf-8'))
f.getvalue()

### 输入输出

* input() 函数暂停程序运行，同时等待键盘输入；直到回车被按下，函数的参数即为提示语，输入类型永远是字符串型（str）
* 对 int 类型没有最大限制（相比之下， C++ 的 int 最大为 2147483647，超过这个数字会产生溢出），但是对 float 类型依然有精度限制

In [ ]:
name = input('your name:')
gender = input('you are a boy?(y/n)')

welcome_str = 'Welcome to the matrix {prefix} {name}.'
welcome_dic = {
    'prefix': 'Mr.' if gender == 'y' else 'Mrs',
    'name': name
}

print('authorizing...')
welcome_str.format(**welcome_dic)

## 简单 NLP（自然语言处理）任务

- 给 read 指定参数 size ，用来表示读取最大长度
- 通过 readline() 函数，每次读取一行，常用于数据挖掘（Data Mining）中数据清洗，在写一些小的程序时非常轻便
- with 语句 不需要显式调用 close()


In [ ]:
import re


def parse(text):
    text = re.sub(r'[^\w]', ' ', text)
    text = text.lower()
    word_list = text.split(' ')
    # 去除空白单词
    word_list = filter(None, word_list)

    # 生成单词和词频的字典
    word_cnt = {}
    for word in word_list:
        if word not in word_cnt:
            word_cnt[word] = 0
        word_cnt[word] += 1
    sorted_word_cnt = sorted(
        word_cnt.items(), key=lambda kv: kv[1], reverse=True)

    return sorted_word_cnt


with open('in.txt', 'r') as fin:
    text = fin.read()

word_and_freq = parse(text)
with open('out.txt', 'w') as fout:
    for word, freq in word_and_freq:
        fout.write('{} {}\n'.format(word, freq))

### 序列化

In [ ]:
import pickle, json

d = dict(name='Bob', age=20, score=88)
with open('../data/content.txt', 'wb') as f:
    pickle.dump(d, f)

with open('../data/content.txt', 'rb') as f:
    d = pickle.load(f)
    print(d)

#### JSON 序列化与实战

In [ ]:
import json

params = {
    'symbol': '123456',
    'type': 'limit',
    'price': 123.4,
    'amount': 23
}

with open('params.json', 'w') as fout:
    params_str = json.dump(params, fout)

print('after json serialization')
print('type of params_str = {}, params_str = {}'.format(type(params_str), params))

with open('params.json', 'r') as fin:
    original_params = json.load(fin)

print('after json deserialization')
print('type of original_params = {}, original_params = {}'.format(type(original_params), original_params))

### context manager

- 任何一门编程语言中文件输入输出、数据库连接断开等都是很常见的资源管理操作
* 资源都是有限的，写程序时必须保证这些资源在使用后得到释放，不然就容易造成资源泄露，轻者使得系统处理缓慢，重则会使系统崩溃
* 上下文管理器 能够帮助自动分配并且释放资源，最典型应用 with 语句
* 基于类实现上下文管理器时，必须保证这个类包括方法
    - __enter__ 返回需要被管理的资源
    - __exit__ 通常会存在一些释放、清理资源操作,参数“exc_type, exc_val, exc_tb”表示 exception_type、exception_value 和 traceback
    - 当执行含有上下文管理器 with 语句时，如果有异常抛出，异常信息会包含在这三个变量中，传入方法“__exit__()
* 基于生成器实现

In [ ]:
for x in range(10000000):
    with open('test.txt', 'w') as f:
        f.write('hello')

In [ ]:
f = open('test.txt', 'w')
try:
    f.write('hello')
finally:
    f.close()

In [ ]:
some_lock = threading.Lock()
some_lock.acquire()
try:
    ...
finally:
    some_lock.release()

In [ ]:

class FileManager:
    def __init__(self, name, mode):
        print('calling __init__ method')
        self.name = name
        self.mode = mode
        self.file = None

    def __enter__(self):
        print('calling __enter__ method')
        self.file = open(self.name, self.mode)
        return self.file


    def __exit__(self, exc_type, exc_val, exc_tb):
        print('calling __exit__ method')
        if exc_type:
            print(f'exc_type: {exc_type}')
            print(f'exc_value: {exc_value}')
            print(f'exc_traceback: {exc_tb}')
            print('exception handled')

        if self.file:
            self.file.close()

with FileManager('test.txt', 'w') as f:
    print('ready to write to file')
    f.write('hello world')

In [ ]:

from contextlib import contextmanager

@contextmanager
def file_manager(name, mode):
    try:
        f = open(name, mode)
        yield f
    finally:
        f.close()

with file_manager('test.txt', 'w') as f:
    f.write('hello world')

##### 思考

* 制定长度，获取到下一个非字符位置，并存储
* 有限容量网盘同步大文件
    * 控制文件加状态：服务端传完，客户端才可以读取
    * 检测文件是否传完

In [ ]:
import re

CHUNK_SIZE = 100  # 一次最多读取的字符长度


# 这个函数每次会接收上一次得到的 last_word，然后和这次的 text 合并起来处理。
# 合并后判断最后一个词有没有可能连续，并分离出来，然后返回。
# 这里的代码没有 if 语句，但是仍然是正确的，可以想一想为什么。
def parse_to_word_list(text, last_word, word_list):
    text = re.sub(r'[^\w ]', ' ', last_word + text)
    text = text.lower()
    cur_word_list = text.split(' ')
    cur_word_list, last_word = cur_word_list[:-1], cur_word_list[-1]
    word_list += filter(None, cur_word_list)

    return last_word


def solve():
    with open('in.txt', 'r') as fin:
        word_list, last_word = [], ''
        while True:
            text = fin.read(CHUNK_SIZE)
            if not text:
                break  # 读取完毕，中断循环
            last_word = parse_to_word_list(text, last_word, word_list)

        word_cnt = {}
        for word in word_list:
            if word not in word_cnt:
                word_cnt[word] = 0
            word_cnt[word] += 1

        sorted_word_cnt = sorted(word_cnt.items(), key=lambda kv: kv[1], reverse=True)
        return sorted_word_cnt


print(solve())

In [ ]:
 # server.py
# 假设 server 电脑上所有的文件都在 BASR_DIR 中，为了简化不考虑文件夹结构，网盘路径在 NET_DIR

import os
from shutil import copyfile
import time

BASE_DIR = 'server/'
NET_DIR = 'net/'


def main():
    filenames = os.listdir(BASE_DIR)
    for i, filename in enumerate(filenames):
        print('copying {} into net drive... {}/{}'.format(filename, i + 1, len(filenames)))
        copyfile(BASE_DIR + filename, NET_DIR + filename)
        print('copied {} into net drive, waiting client complete... {}/{}'.format(filename, i + 1, len(filenames)))

        while os.path.exists(NET_DIR + filename):
            time.sleep(3)

    print('transferred {} into client. {}/{}'.format(filename, i + 1, len(filenames)))


if __name__ == "__main__":
    main()

In [ ]:
# client.py
# 假设 client 输出文件夹 BASR_DIR ，网盘路径 NET_DIR

import os
from shutil import copyfile
import time

BASE_DIR = 'client/'
NET_DIR = 'net/'


def main():
    while True:
        filenames = os.listdir(NET_DIR)
        for filename in filenames:
            print('downloading {} into local disk...'.format(filename))
            copyfile(NET_DIR + filename, BASE_DIR + filename)
            os.remove(NET_DIR + filename)  # 我们需要删除这个文件，网盘会提我们同步这个操作，从而 server 知晓已完成
            print('downloaded {} into local disk.'.format(filename))
        time.sleep(3)


if __name__ == "__main__":
    main()